# Population interactions

## Starting assumptions
So we've now laboriously got to the point that we should have
some idea of what we are meaning by the mixing matrices
that we might like to put into a model.
But what to we really mean epidemiologically by
different rates of contact between population groups?

Suppose we have two population groups,
and we want to consider that people from one population group
just has less social activity than another
and comes into contact with fewer people than the other one.
If this is the way we're thinking,
we really need a matrix to represent even this simple assumption,
because contacts work in both directions.
That is, if the susceptible individual from the one modelled group 
is less likely to be contacted by other individuals 
in the population because she/he is less socially active,
then it is likely that she/he is also less likely to contact
other persons in the population too.

Let's start looking at this under the density-dependent framework,
because we're dealing with _per capita per capita_ rates 
considering both the susceptible and the infecting individual.
In this case, we might like to consider that the rate at which 
the first population group contacts the second is the same as the rate
at which the second contacts the first.
Provided we are thinking of the contents of the mixing matrix as 
rates of contact per unit time, this is fine.
If we also want to consider that one population group is more susceptible
or more infectious than the other, we could do this by adjusting
the infection flow rate (to change susceptibility) or
the infectiousness of one of these categories,
without adjusting the mixing matrix - 
or better yet, just assume everyone is equally susceptible and infectious.
We can also take the risk of infection per contact idea out of the equation
by thinking of the parameter for the infection transition rate 
as this quantity.
That is, to keep things as simple as possible,
let's think of our mixing matrix as containing information
on the rate of social contact between the two specific population
sub-groups we are simulating.

To summarise, let's start off assuming:
- Two interacting population subgroups
- Density dependence
- Uniform susceptibility and infectiousness
- Our mixing matrix contains information on the rate of interaction between pairs of individuals from the two population subgroups
represented by our matrix

## Assortative mixing
A very common assumption in infectious disease modelling is that
people in our population who share certain characteristics
are more likely to come into contact with one-another 
than people who have different characteristics.
For example, our two population groups might represent
people living in two neighbouring towns,
and we might want to consider that the rate at which
people come into contact with other people from their own town
is greater than the rate at which they come into contact with the other town.

Let's build a really simple matrix to represent this.

In [ ]:
import numpy as np
import pandas as pd
from jax import numpy as jnp
pd.options.plotting.backend = "plotly"
import plotly.express as px

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, Function

In [ ]:
assortative_matrix = np.array(
    [
        [1.0, 0.5], 
        [0.5, 1.0],
    ]
)

px.imshow(
    assortative_matrix,
    title="Assortative mixing matrix",
)

In [ ]:
def build_sir_model(
    config: dict,
) -> CompartmentalModel:
    """
    This is the same model as introduced in the mixing-and-transmission notebook
    
    Args:
        config: User requests to define model construction
    Returns:
        The model object, that won't do much yet       
    """
    
    # Model characteristics
    compartments = config["compartments"]
    analysis_times = (0.0, config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        }
    )
    
    # Transitions
    model.add_infection_density_flow(
        name="infection",
        contact_rate=Parameter("risk_per_contact"),
        source="susceptible",
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / Parameter("infectious_period"),
        source="infectious", 
        dest="recovered",
    )
    
    # Output
    model.request_output_for_compartments(
        "prevalence",
        "infectious",
    )
    
    return model

In [ ]:
def build_frequency_mixing_matrix(intergroup_contact_rate, jax=True):
    values = [
        [1.0, intergroup_contact_rate],
        [intergroup_contact_rate, 1.0],
    ]
    
    return jnp.array(values) if jax else np.array(values)

In [ ]:
def build_simple_strat(
    compartments: list,
) -> Stratification:
    """
    Get a stratification that divides the population into two groups
    
    Args:
        compartments: The compartments to be stratified (here all the model's compartments)
        mixing_matrix: The mixing matrix for this stratification
    Returns:
        The completed Stratification object    
    """
                
    mix_strat = Stratification(
        "groups",
        ["group1", "group2"],
        compartments,
    )
    
    prop1 = Parameter("prop1")
    prop2 = 1. - prop1
    mix_strat.set_population_split(
        {
            "group1": prop1,
            "group2": prop2,
        }
    )

    mixing_matrix = Function(
        build_frequency_mixing_matrix, 
        (Parameter("intergroup_interaction"),),
    )
    
    mix_strat.set_mixing_matrix(mixing_matrix)

    return mix_strat

In [ ]:
model_config = {
    "end_time": 20.0,
    "population": 1.0,
    "seed": 0.01,
    "compartments": ("susceptible", "infectious", "recovered"),
}

parameters = {
    "risk_per_contact": 1.0,
    "infectious_period": 2.0,
    "prop1": 0.5,
    "intergroup_interaction": 1.0,
}

In [ ]:
simple_model = build_sir_model(model_config)
mix_strat = build_simple_strat(model_config["compartments"])
simple_model.stratify_with(mix_strat)
simple_model.run(parameters=parameters)

At this stage, the model is very simple
and the stratification isn't doing anything 
because nothing has been adjusted and the mixing matrix only contains ones.
The $R_{0}$ of this model is two,
because the average infectious period is two days,
and infectious people infect one person per day infectious
in a fully susceptible population.
However, for heterogeneous mixing models,
the $R_{0}$ is more generally defined by the dominant Eigenvalue of the matrix
multiplied through by the transmission probability.

In [ ]:
evalue, _ = np.linalg.eig(build_frequency_mixing_matrix(parameters["intergroup_interaction"], jax=False))
evalue.max() * parameters["risk_per_contact"]

We'll now introduce a matrix that implements assortative mixing,
by setting the off-diagonal elements of the matrix to a lower value than one.
However, we can still easily recover the same behaviour
by scaling the transmission risk up to account for the lower dominant
Eigenvalue of this matrix.

In [ ]:
parameters.update(
    {    
        "intergroup_interaction": 0.5,
        "risk_per_contact": 4.0 / 3.0,  # Scale up so that the dominant eigenvalue is the same
    }
)
evalue, _ = np.linalg.eig(build_frequency_mixing_matrix(parameters["intergroup_interaction"], jax=False))
evalue.max() * parameters["risk_per_contact"]

In [ ]:
hetero_mix_model = build_sir_model(model_config)
mix_strat = build_simple_strat(model_config["compartments"])
hetero_mix_model.stratify_with(mix_strat)
hetero_mix_model.run(parameters=parameters)

In [ ]:
pd.DataFrame(
    {
        "simple": simple_model.get_derived_outputs_df()["prevalence"],
        "hetero": hetero_mix_model.get_derived_outputs_df()["prevalence"],
    }
).plot()